In [1]:
import tensorflow as tf
print('TensorFlow Version: {}'.format(tf.__version__))
print('Default GPU Device: {}'.format(tf.test.gpu_device_name()))

TensorFlow Version: 1.7.1
Default GPU Device: 


##### >**Note:** Make sure you have OpenAI Gym cloned. Then run this command `pip install -e gym/[all]`.

In [2]:
import numpy as np
import gym
env = gym.make('CartPole-v0')
env = gym.make('CartPole-v1')

In [3]:
def model_input(state_size):
    #states = tf.placeholder(tf.float32, [None, *state_size], name='states')
    states = tf.placeholder(tf.float32, [None, state_size], name='states')
    actions = tf.placeholder(tf.int32, [None], name='actions')
    next_states = tf.placeholder(tf.float32, [None, state_size], name='next_states')
    rewards = tf.placeholder(tf.float32, [None], name='rewards')
    dones = tf.placeholder(tf.float32, [None], name='dones')
    rates = tf.placeholder(tf.float32, [None], name='rates') # success rate
    return states, actions, next_states, rewards, dones, rates

In [4]:
def actor(states, action_size, hidden_size, reuse=False, alpha=0.1, training=False):
    with tf.variable_scope('actor', reuse=reuse):
        # First fully connected layer
        h1 = tf.layers.dense(inputs=states, units=hidden_size)
        bn1 = tf.layers.batch_normalization(h1, training=training)        
        nl1 = tf.maximum(alpha * bn1, bn1)
        
        # Second fully connected layer
        h2 = tf.layers.dense(inputs=nl1, units=hidden_size)
        bn2 = tf.layers.batch_normalization(h2, training=training)        
        nl2 = tf.maximum(alpha * bn2, bn2)
        
        # Output layer
        logits = tf.layers.dense(inputs=nl2, units=action_size)        
        return logits

In [5]:
def generator(actions, state_size, hidden_size, reuse=False, alpha=0.1, training=False):
    with tf.variable_scope('generator', reuse=reuse):
        # First fully connected layer
        h1 = tf.layers.dense(inputs=actions, units=hidden_size)
        bn1 = tf.layers.batch_normalization(h1, training=training)        
        nl1 = tf.maximum(alpha * bn1, bn1)
        
        # Second fully connected layer
        h2 = tf.layers.dense(inputs=nl1, units=hidden_size)
        bn2 = tf.layers.batch_normalization(h2, training=training)        
        nl2 = tf.maximum(alpha * bn2, bn2)
        
        # Output layer
        logits = tf.layers.dense(inputs=nl2, units=state_size)        
        return logits

In [6]:
def discriminator(states, actions, action_size, hidden_size, reuse=False, alpha=0.1, training=False):
    with tf.variable_scope('discriminator', reuse=reuse):
        # First fully connected layer
        h1 = tf.layers.dense(inputs=states, units=action_size)
        bn1 = tf.layers.batch_normalization(h1, training=training)        
        nl1 = tf.maximum(alpha * bn1, bn1)
        
        # Second fully connected layer
        nl1_fused = tf.concat(axis=1, values=[nl1, actions])
        h2 = tf.layers.dense(inputs=nl1_fused, units=hidden_size)
        bn2 = tf.layers.batch_normalization(h2, training=training)        
        nl2 = tf.maximum(alpha * bn2, bn2)
                
        # Output layer
        logits = tf.layers.dense(inputs=nl2, units=1)        
        return logits

In [16]:
def model_loss(state_size, action_size, hidden_size, gamma,
               states, actions, next_states, rewards, dones, rates):
    actions_logits = actor(states=states, hidden_size=hidden_size, action_size=action_size)
    actions_labels = tf.one_hot(indices=actions, depth=action_size, dtype=actions_logits.dtype)
    aloss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(logits=actions_logits, 
                                                                      labels=actions_labels))
    ###############################################
    next_states_logits = generator(actions=actions_logits, hidden_size=hidden_size, state_size=state_size)
    next_states_labels = tf.nn.sigmoid(next_states)
    aloss += tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=next_states_logits, 
                                                                    labels=next_states_labels))
    ####################################################
    dQs = discriminator(actions=actions_labels, hidden_size=hidden_size, states=states, action_size=action_size)
    rates = tf.reshape(rates, shape=[-1, 1])
    dloss = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=dQs, # GAN
                                                                   labels=rates)) # 0-1
    ####################################################
    gQs = discriminator(actions=actions_logits, hidden_size=hidden_size, states=states, action_size=action_size, 
                        reuse=True)
    dloss += tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=gQs, # GAN
                                                                    labels=tf.zeros_like(gQs))) # 0-1
    aloss2 = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=gQs, # GAN
                                                                    labels=tf.ones_like(gQs))) # 0-1
    #####################################################
    next_actions_logits = actor(states=next_states, hidden_size=hidden_size, action_size=action_size, reuse=True)
    gQs2 = discriminator(actions=next_actions_logits, hidden_size=hidden_size, states=next_states, 
                         action_size=action_size, reuse=True)
    gQs2 = tf.reshape(gQs2, shape=[-1]) * dones
    aloss2 += tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=gQs2, # GAN
                                                                     labels=tf.ones_like(gQs2))) # 0-1
    #####################################################
    next_states_logits = generator(actions=next_actions_logits, hidden_size=hidden_size, state_size=state_size, 
                                   reuse=True)
    next_actions_logits = actor(states=next_states_logits, hidden_size=hidden_size, action_size=action_size, 
                                reuse=True)
    gQs3 = discriminator(actions=next_actions_logits, hidden_size=hidden_size, states=next_states_logits, 
                         action_size=action_size, reuse=True)
    dones = tf.concat(axis=0, values=[dones[1:], tf.zeros(shape=[1])])
    gQs3 = tf.reshape(gQs3, shape=[-1]) * dones
    aloss2 += tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=gQs3, # GAN
                                                                     labels=tf.ones_like(gQs3))) # 0-1
    #####################################################
    next_states_logits = generator(actions=next_actions_logits, hidden_size=hidden_size, state_size=state_size, 
                                   reuse=True)
    next_actions_logits = actor(states=next_states_logits, hidden_size=hidden_size, action_size=action_size, 
                                reuse=True)
    gQs3 = discriminator(actions=next_actions_logits, hidden_size=hidden_size, states=next_states_logits, 
                         action_size=action_size, reuse=True)
    dones = tf.concat(axis=0, values=[dones[1:], tf.zeros(shape=[1])])
    gQs3 = tf.reshape(gQs3, shape=[-1]) * dones
    aloss2 += tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=gQs3, # GAN
                                                                     labels=tf.ones_like(gQs3))) # 0-1
    return actions_logits, aloss, dloss, aloss2

In [17]:
def model_opt(a_loss, a_loss2, d_loss, a_learning_rate, d_learning_rate):
    # Get weights and bias to update
    t_vars = tf.trainable_variables()
    a_vars = [var for var in t_vars if var.name.startswith('actor')]
    g_vars = [var for var in t_vars if var.name.startswith('generator')]
    d_vars = [var for var in t_vars if var.name.startswith('discriminator')]

    # Optimize
    with tf.control_dependencies(tf.get_collection(tf.GraphKeys.UPDATE_OPS)): # Required for batchnorm (BN)
        a_opt = tf.train.AdamOptimizer(a_learning_rate).minimize(a_loss, var_list=a_vars)
        d_opt = tf.train.AdamOptimizer(d_learning_rate).minimize(d_loss, var_list=d_vars)
        a_opt2 = tf.train.AdamOptimizer(a_learning_rate).minimize(a_loss2, var_list=a_vars)
    return a_opt, d_opt, a_opt2

In [18]:
class Model:
    def __init__(self, state_size, action_size, hidden_size, a_learning_rate, d_learning_rate, gamma):

        # Data of the Model: make the data available inside the framework
        self.states, self.actions, self.next_states, self.rewards, self.dones, self.rates = model_input(
            state_size=state_size)

        # Create the Model: calculating the loss and forwad pass
        self.actions_logits, self.a_loss, self.d_loss, self.a_loss2 = model_loss(
            state_size=state_size, action_size=action_size, hidden_size=hidden_size, gamma=gamma, # model init
            states=self.states, actions=self.actions, next_states=self.next_states, #model input 
            rewards=self.rewards, dones=self.dones, rates=self.rates) # model input
        
        # Update the model: backward pass and backprop
        self.a_opt, self.d_opt, self.a_opt2 = model_opt(a_loss=self.a_loss, 
                                                        a_loss2=self.a_loss2, 
                                                        d_loss=self.d_loss,
                                                        a_learning_rate=a_learning_rate,
                                                        d_learning_rate=d_learning_rate)

In [19]:
# def sample(buffer, batch_size):
#     idx = np.random.choice(np.arange(len(buffer)), size=batch_size, replace=False)
#     return [buffer[ii] for ii in idx]

In [20]:
from collections import deque
class Memory():
    def __init__(self, max_size = 1000):
        self.buffer = deque(maxlen=max_size) # data batch

## Hyperparameters

One of the more difficult aspects of reinforcememt learning are the large number of hyperparameters. Not only are we tuning the network, but we're tuning the simulation.

In [21]:
env.observation_space, env.action_space

(Box(4,), Discrete(2))

In [22]:
# Exploration parameters
explore_start = 1.0            # exploration probability at start
explore_stop = 0.01           # minimum exploration probability 
decay_rate = 0.0001            # exponential decay rate for exploration prob

# Network parameters
state_size = 4
action_size = 2
hidden_size = 4*2             # number of units in each Q-network hidden layer
a_learning_rate = 1e-4         # Q-network learning rate
d_learning_rate = 1e-4         # Q-network learning rate

# Memory parameters
memory_size = int(1e5)            # memory capacity
batch_size = int(1e3)             # experience mini-batch size: 200/500 a successfull episode size
gamma = 0.99                   # future reward discount

In [23]:
# Reset/init the graph/session
graph = tf.reset_default_graph()

# Init the model
model = Model(action_size=action_size, state_size=state_size, hidden_size=hidden_size,
              a_learning_rate=a_learning_rate, d_learning_rate=d_learning_rate, gamma=gamma)

# Init the memory
memory = Memory(max_size=memory_size)

In [24]:
state = env.reset()
total_reward = 0
num_step = 0
for _ in range(memory_size):
    action = env.action_space.sample()
    next_state, reward, done, _ = env.step(action)
    rate = -1
    memory.buffer.append([state, action, next_state, reward, float(done), rate])
    num_step += 1 # memory incremented
    total_reward += reward
    state = next_state
    if done is True:
        state = env.reset()
        rate = total_reward/500
        for idx in range(num_step): # episode length
            if memory.buffer[-1-idx][-1] == -1:
                memory.buffer[-1-idx][-1] = rate
        total_reward = 0 # reset
        num_step = 0 # reset

## Training the model

Below we'll train our agent. If you want to watch it train, uncomment the `env.render()` line. This is slow because it's rendering the frames slower than the network can train. But, it's cool to watch the agent get better at the game.

In [ ]:
# Save/load the model and save for plotting
saver = tf.train.Saver()
episode_rewards_list, rewards_list = [], []
# aloss_list, dloss_list, aloss2_list = [], []

# TF session for training
with tf.Session(graph=graph) as sess:
    sess.run(tf.global_variables_initializer())
    #saver.restore(sess, 'checkpoints/model.ckpt')    
    #saver.restore(sess, tf.train.latest_checkpoint('checkpoints'))
    total_step = 0 # Explore or exploit parameter
    episode_reward = deque(maxlen=100) # 100 episodes average/running average/running mean/window

    # Training episodes/epochs
    for ep in range(11111):
        total_reward = 0 # each episode
        aloss_batch, dloss_batch, aloss2_batch = [], [], []
        state = env.reset() # each episode
        num_step = 0 # each episode

        # Training steps/batches
        while True:
            # Explore (Env) or Exploit (Model)
            total_step += 1
            explore_p = explore_stop + (explore_start - explore_stop) * np.exp(-decay_rate * total_step) 
            if explore_p > np.random.rand():
                action = env.action_space.sample()
            else:
                action_logits = sess.run(model.actions_logits, feed_dict={model.states: state.reshape([1, -1])})
                action = np.argmax(action_logits)
            next_state, reward, done, _ = env.step(action)
            rate = -1
            memory.buffer.append([state, action, next_state, reward, float(done), rate])
            num_step += 1 # momory added
            total_reward += reward
            state = next_state
            
            # Rating the memory
            if done is True:
                rate = total_reward/500 # update rate at the end/ when episode is done
                for idx in range(num_step): # episode length
                    if memory.buffer[-1-idx][-1] == -1: # double-check the landmark/marked indexes
                        memory.buffer[-1-idx][-1] = rate # rate the trajectory/data
                        
            # Training with the maxrated minibatch
            batch = memory.buffer
            #for idx in range(memory_size// batch_size):
            idx = np.random.choice(np.arange(memory_size// batch_size))
            states = np.array([each[0] for each in batch])[idx*batch_size:(idx+1)*batch_size]
            actions = np.array([each[1] for each in batch])[idx*batch_size:(idx+1)*batch_size]
            next_states = np.array([each[2] for each in batch])[idx*batch_size:(idx+1)*batch_size]
            rewards = np.array([each[3] for each in batch])[idx*batch_size:(idx+1)*batch_size]
            dones = np.array([each[4] for each in batch])[idx*batch_size:(idx+1)*batch_size]
            rates = np.array([each[5] for each in batch])[idx*batch_size:(idx+1)*batch_size]
            states = states[rates >= np.max(rates)]
            actions = actions[rates >= np.max(rates)]
            next_states = next_states[rates >= np.max(rates)]
            rewards = rewards[rates >= np.max(rates)]
            dones = dones[rates >= np.max(rates)]
            rates = rates[rates >= np.max(rates)]
            aloss, dloss, _, _ = sess.run([model.a_loss, model.d_loss, model.a_opt, model.d_opt],
                                          feed_dict = {model.states: states, 
                                                       model.actions: actions,
                                                       model.next_states: next_states,
                                                       model.rewards: rewards,
                                                       model.dones: dones,
                                                       model.rates: rates})
            aloss2, _ = sess.run([model.a_loss2, model.a_opt2], 
                                 feed_dict = {model.states: states, 
                                              model.actions: actions,
                                              model.next_states: next_states,
                                              model.rewards: rewards,
                                              model.dones: dones,
                                              model.rates: rates})
            aloss_batch.append(aloss)
            dloss_batch.append(dloss)
            aloss2_batch.append(aloss2)
            if done is True:
                break
                
        episode_reward.append(total_reward)
        print('Episode:{}'.format(ep),
              'meanR:{:.4f}'.format(np.mean(episode_reward)),
              'R:{:.4f}'.format(total_reward),
              'rate:{:.4f}'.format(rate),
              'aloss:{:.4f}'.format(np.mean(aloss_batch)),
              'dloss:{:.4f}'.format(np.mean(dloss_batch)),
              'aloss2:{:.4f}'.format(np.mean(aloss2_batch)),
              'exploreP:{:.4f}'.format(explore_p))

        # Ploting out
        episode_rewards_list.append([ep, np.mean(episode_reward)])
        rewards_list.append([ep, total_reward])
        # gloss_list.append([ep, np.mean(gloss_batch)])
        # dloss_list.append([ep, np.mean(dloss_batch)])
        
        # Break episode/epoch loop
        ## Option 1: Solve the First Version
        #The task is episodic, and in order to solve the environment, 
        #your agent must get an average score of +30 over 100 consecutive episodes.
        if np.mean(episode_reward) >= 500:
            break
            
    # At the end of all training episodes/epochs
    saver.save(sess, 'checkpoints/model.ckpt')

Episode:0 meanR:15.0000 R:15.0000 rate:0.0300 aloss:1.3869 dloss:1.3953 aloss2:2.7795 exploreP:0.9985
Episode:1 meanR:17.5000 R:20.0000 rate:0.0400 aloss:1.3868 dloss:1.3831 aloss2:2.7873 exploreP:0.9965
Episode:2 meanR:16.6667 R:15.0000 rate:0.0300 aloss:1.3868 dloss:1.3722 aloss2:2.7920 exploreP:0.9951
Episode:3 meanR:16.0000 R:14.0000 rate:0.0280 aloss:1.3857 dloss:1.3629 aloss2:2.7931 exploreP:0.9937
Episode:4 meanR:17.2000 R:22.0000 rate:0.0440 aloss:1.3849 dloss:1.3508 aloss2:2.8007 exploreP:0.9915
Episode:5 meanR:17.8333 R:21.0000 rate:0.0420 aloss:1.3849 dloss:1.3397 aloss2:2.8059 exploreP:0.9895
Episode:6 meanR:17.7143 R:17.0000 rate:0.0340 aloss:1.3856 dloss:1.3273 aloss2:2.8113 exploreP:0.9878
Episode:7 meanR:18.0000 R:20.0000 rate:0.0400 aloss:1.3850 dloss:1.3195 aloss2:2.8130 exploreP:0.9858
Episode:8 meanR:17.5556 R:14.0000 rate:0.0280 aloss:1.3846 dloss:1.3071 aloss2:2.8239 exploreP:0.9845
Episode:9 meanR:20.5000 R:47.0000 rate:0.0940 aloss:1.3832 dloss:1.2919 aloss2:2.8

Episode:80 meanR:23.0000 R:22.0000 rate:0.0440 aloss:1.3639 dloss:0.6007 aloss2:3.9114 exploreP:0.8317
Episode:81 meanR:23.1707 R:37.0000 rate:0.0740 aloss:1.3674 dloss:0.5584 aloss2:3.9580 exploreP:0.8287
Episode:82 meanR:23.1566 R:22.0000 rate:0.0440 aloss:1.3651 dloss:0.5878 aloss2:4.0371 exploreP:0.8269
Episode:83 meanR:23.1190 R:20.0000 rate:0.0400 aloss:1.3653 dloss:0.5545 aloss2:4.1211 exploreP:0.8253
Episode:84 meanR:23.2118 R:31.0000 rate:0.0620 aloss:1.3663 dloss:0.5594 aloss2:3.9875 exploreP:0.8227
Episode:85 meanR:23.1395 R:17.0000 rate:0.0340 aloss:1.3603 dloss:0.5353 aloss2:4.1549 exploreP:0.8214
Episode:86 meanR:23.0575 R:16.0000 rate:0.0320 aloss:1.3686 dloss:0.5628 aloss2:4.0700 exploreP:0.8201
Episode:87 meanR:23.1364 R:30.0000 rate:0.0600 aloss:1.3723 dloss:0.5370 aloss2:4.0645 exploreP:0.8176
Episode:88 meanR:23.0562 R:16.0000 rate:0.0320 aloss:1.3624 dloss:0.5467 aloss2:4.0814 exploreP:0.8163
Episode:89 meanR:23.0000 R:18.0000 rate:0.0360 aloss:1.3595 dloss:0.5626 

Episode:159 meanR:39.3500 R:15.0000 rate:0.0300 aloss:1.3952 dloss:0.4708 aloss2:4.9125 exploreP:0.5935
Episode:160 meanR:39.6300 R:45.0000 rate:0.0900 aloss:1.4080 dloss:0.4689 aloss2:4.9050 exploreP:0.5909
Episode:161 meanR:40.2700 R:85.0000 rate:0.1700 aloss:1.3978 dloss:0.4732 aloss2:4.9581 exploreP:0.5859
Episode:162 meanR:40.7000 R:63.0000 rate:0.1260 aloss:1.3988 dloss:0.4886 aloss2:4.9749 exploreP:0.5823
Episode:163 meanR:40.4900 R:17.0000 rate:0.0340 aloss:1.4137 dloss:0.4557 aloss2:4.9813 exploreP:0.5814
Episode:164 meanR:41.1900 R:85.0000 rate:0.1700 aloss:1.4075 dloss:0.4736 aloss2:4.9386 exploreP:0.5765
Episode:165 meanR:41.4800 R:46.0000 rate:0.0920 aloss:1.3974 dloss:0.4926 aloss2:4.9271 exploreP:0.5739
Episode:166 meanR:41.7100 R:48.0000 rate:0.0960 aloss:1.4328 dloss:0.4644 aloss2:4.9507 exploreP:0.5712
Episode:167 meanR:42.1400 R:76.0000 rate:0.1520 aloss:1.4007 dloss:0.4797 aloss2:4.9500 exploreP:0.5670
Episode:168 meanR:42.1400 R:27.0000 rate:0.0540 aloss:1.3942 dlo

Episode:238 meanR:63.8600 R:165.0000 rate:0.3300 aloss:1.4189 dloss:0.4891 aloss2:4.9386 exploreP:0.3469
Episode:239 meanR:64.4700 R:97.0000 rate:0.1940 aloss:1.4027 dloss:0.4699 aloss2:4.9148 exploreP:0.3437
Episode:240 meanR:64.7700 R:68.0000 rate:0.1360 aloss:1.3973 dloss:0.5180 aloss2:4.8896 exploreP:0.3414
Episode:241 meanR:64.7100 R:47.0000 rate:0.0940 aloss:1.4029 dloss:0.4936 aloss2:4.8816 exploreP:0.3399
Episode:242 meanR:65.0800 R:59.0000 rate:0.1180 aloss:1.4047 dloss:0.5072 aloss2:4.8590 exploreP:0.3379
Episode:243 meanR:65.3900 R:103.0000 rate:0.2060 aloss:1.4047 dloss:0.4867 aloss2:4.8682 exploreP:0.3346
Episode:244 meanR:65.8900 R:65.0000 rate:0.1300 aloss:1.4155 dloss:0.4985 aloss2:4.8805 exploreP:0.3325
Episode:245 meanR:66.0200 R:72.0000 rate:0.1440 aloss:1.4001 dloss:0.5037 aloss2:4.8744 exploreP:0.3302
Episode:246 meanR:66.3300 R:65.0000 rate:0.1300 aloss:1.4021 dloss:0.5259 aloss2:4.8744 exploreP:0.3281
Episode:247 meanR:66.7500 R:62.0000 rate:0.1240 aloss:1.3965 d

Episode:317 meanR:83.4200 R:41.0000 rate:0.0820 aloss:1.3900 dloss:0.5360 aloss2:4.7946 exploreP:0.1873
Episode:318 meanR:84.0000 R:116.0000 rate:0.2320 aloss:1.3869 dloss:0.5613 aloss2:4.7429 exploreP:0.1852
Episode:319 meanR:84.8700 R:103.0000 rate:0.2060 aloss:1.3913 dloss:0.5442 aloss2:4.7487 exploreP:0.1834
Episode:320 meanR:84.8900 R:68.0000 rate:0.1360 aloss:1.3925 dloss:0.5300 aloss2:4.7548 exploreP:0.1823
Episode:321 meanR:84.6600 R:89.0000 rate:0.1780 aloss:1.4005 dloss:0.5561 aloss2:4.7540 exploreP:0.1807
Episode:322 meanR:84.6400 R:97.0000 rate:0.1940 aloss:1.3986 dloss:0.4917 aloss2:4.7638 exploreP:0.1791
Episode:323 meanR:84.4900 R:56.0000 rate:0.1120 aloss:1.4123 dloss:0.4812 aloss2:4.8024 exploreP:0.1781
Episode:324 meanR:84.3400 R:54.0000 rate:0.1080 aloss:1.4029 dloss:0.5347 aloss2:4.8039 exploreP:0.1772
Episode:325 meanR:84.8700 R:128.0000 rate:0.2560 aloss:1.3896 dloss:0.5428 aloss2:4.7872 exploreP:0.1751
Episode:326 meanR:85.2400 R:111.0000 rate:0.2220 aloss:1.3913

# Visualizing training

Below I'll plot the total rewards for each episode. I'm plotting the rolling average too, in blue.

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline

def running_mean(x, N):
    cumsum = np.cumsum(np.insert(x, 0, 0)) 
    return (cumsum[N:] - cumsum[:-N]) / N 

In [ ]:
eps, arr = np.array(episode_rewards_list).T
smoothed_arr = running_mean(arr, 10)
plt.plot(eps[-len(smoothed_arr):], smoothed_arr)
plt.plot(eps, arr, color='grey', alpha=0.3)
plt.xlabel('Episode')
plt.ylabel('Total rewards')

In [ ]:
eps, arr = np.array(rewards_list).T
smoothed_arr = running_mean(arr, 10)
plt.plot(eps[-len(smoothed_arr):], smoothed_arr)
plt.plot(eps, arr, color='grey', alpha=0.3)
plt.xlabel('Episode')
plt.ylabel('Total rewards')

In [ ]:
eps, arr = np.array(gloss_list).T
smoothed_arr = running_mean(arr, 10)
plt.plot(eps[-len(smoothed_arr):], smoothed_arr)
plt.plot(eps, arr, color='grey', alpha=0.3)
plt.xlabel('Episode')
plt.ylabel('G losses')`

In [ ]:
eps, arr = np.array(dloss_list).T
smoothed_arr = running_mean(arr, 10)
plt.plot(eps[-len(smoothed_arr):], smoothed_arr)
plt.plot(eps, arr, color='grey', alpha=0.3)
plt.xlabel('Episode')
plt.ylabel('D losses')

## Testing

Let's checkout how our trained agent plays the game.

In [24]:
import gym
env = gym.make('CartPole-v0')
env = gym.make('CartPole-v1')
# env = gym.make('Acrobot-v1')
# env = gym.make('MountainCar-v0')
# env = gym.make('Pendulum-v0')
# env = gym.make('Blackjack-v0')
# env = gym.make('FrozenLake-v0')
# env = gym.make('AirRaid-ram-v0')
# env = gym.make('AirRaid-v0')
# env = gym.make('BipedalWalker-v2')
# env = gym.make('Copy-v0')
# env = gym.make('CarRacing-v0')
# env = gym.make('Ant-v2') #mujoco
# env = gym.make('FetchPickAndPlace-v1') # mujoco required!

with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    saver.restore(sess, 'checkpoints/model.ckpt')    
    #saver.restore(sess, tf.train.latest_checkpoint('checkpoints'))
    
    # Episodes/epochs
    for _ in range(10):
        state = env.reset()
        total_reward = 0

        # Steps/batches
        while True:
            env.render()
            action_logits = sess.run(model.actions_logits, feed_dict={model.states: state.reshape([1, -1])})
            action = np.argmax(action_logits)
            state, reward, done, _ = env.step(action)
            total_reward += reward
            if done:
                print('total_reward: {}'.format(total_reward))
                break
                
env.close()

INFO:tensorflow:Restoring parameters from checkpoints/model.ckpt
total_reward: 500.0
total_reward: 500.0
total_reward: 500.0
total_reward: 500.0
total_reward: 481.0
total_reward: 481.0
total_reward: 500.0
total_reward: 500.0
total_reward: 500.0
total_reward: 500.0


## Extending this

So, Cart-Pole is a pretty simple game. However, the same model can be used to train an agent to play something much more complicated like Pong or Space Invaders. Instead of a state like we're using here though, you'd want to use convolutional layers to get the state from the screen images.

![Deep Q-Learning Atari](assets/atari-network.png)

I'll leave it as a challenge for you to use deep Q-learning to train an agent to play Atari games. Here's the original paper which will get you started: http://www.davidqiu.com:8888/research/nature14236.pdf.